# `LinearRegionSelector` with single lines

In [ ]:
import fastplotlib as fpl
import numpy as np
from ipywidgets import IntRangeSlider, FloatRangeSlider, VBox

gp = fpl.GridPlot((2, 2))

# preallocated size for zoomed data
zoomed_prealloc = 1_000

# data to plot
xs = np.linspace(0, 100, 1_000)
sine = np.sin(xs) * 20

# make sine along x axis
sine_graphic_x = gp[0, 0].add_line(sine)

# just something that looks different for line along y-axis
sine_y = sine
sine_y[sine_y > 0] = 0

# sine along y axis
sine_graphic_y = gp[0, 1].add_line(np.column_stack([sine_y, xs]))

# offset the position of the graphic to demonstrate `get_selected_data()` later
sine_graphic_y.position_x = 50
sine_graphic_y.position_y = 50

# add linear selectors
ls_x = sine_graphic_x.add_linear_region_selector()  # default axis is "x"
ls_y = sine_graphic_y.add_linear_region_selector(axis="y")

# preallocate array for storing zoomed in data
zoomed_init = np.column_stack([np.arange(zoomed_prealloc), np.random.rand(zoomed_prealloc)])

# make line graphics for displaying zoomed data
zoomed_x = gp[1, 0].add_line(zoomed_init)
zoomed_y = gp[1, 1].add_line(zoomed_init)


def interpolate(subdata: np.ndarray, axis: int):
    """1D interpolation to display within the preallocated data array"""
    x = np.arange(0, zoomed_prealloc)
    xp = np.linspace(0, zoomed_prealloc, subdata.shape[0])
    
    # interpolate to preallocated size
    return np.interp(x, xp, fp=subdata[:, axis])  # use the y-values


def set_zoom_x(ev):
    """sets zoomed x selector data"""
    selected_data = ev.pick_info["selected_data"]
    zoomed_x.data = interpolate(selected_data, axis=1)  # use the y-values
    gp[1, 0].auto_scale()


def set_zoom_y(ev):
    """sets zoomed y selector data"""
    selected_data = ev.pick_info["selected_data"]
    zoomed_y.data = -interpolate(selected_data, axis=0)  # use the x-values
    gp[1, 1].auto_scale()


# update zoomed plots when bounds change
ls_x.selection.add_event_handler(set_zoom_x)
ls_y.selection.add_event_handler(set_zoom_y)

gp.show(sidecar=False)

### On the x-axis we have a 1-1 mapping from the data that we have passed and the line geometry positions. So the `bounds` min max corresponds directly to the data indices.

In [ ]:
ls_x.selection()

In [ ]:
ls_x.get_selected_indices()

### However, for the y-axis line we have passed a 2D array where we've used a linspace, so there is not a 1-1 mapping from the data to the line geometry positions. Use `get_selected_indices()` to get the indices of the data bounded by the current selection. In addition the position of the Graphic is not `(0, 0)`. You must use `get_selected_indices()` whenever you want the indices of the selected data.

In [ ]:
ls_y.selection()

In [ ]:
ls_y.get_selected_indices()

In [ ]:
import fastplotlib as fpl
import numpy as np

# data to plot
xs = np.linspace(0, 100, 1_000)
sine = np.sin(xs) * 20
cosine = np.cos(xs) * 20

plot = fpl.GridPlot((5, 1))

# sines and cosines
sines = [sine] * 2
cosines = [cosine] * 2

# make line stack
line_stack = plot[0, 0].add_line_stack(sines + cosines, separation=50)

# make selector
selector = line_stack.add_linear_region_selector()

# populate subplots with preallocated graphics
for i, subplot in enumerate(plot):
    if i == 0:
        # skip the first one
        continue
    # make line graphics for displaying zoomed data
    subplot.add_line(zoomed_init, name="zoomed")


def update_zoomed_subplots(ev):
    """update the zoomed subplots"""
    zoomed_data = selector.get_selected_data()
    
    for i in range(len(zoomed_data)):
        data = interpolate(zoomed_data[i], axis=1)
        plot[i + 1, 0]["zoomed"].data = data
        plot[i + 1, 0].auto_scale()


selector.selection.add_event_handler(update_zoomed_subplots)
plot.show(sidecar=False)

# Large line stack with selector

In [ ]:
import fastplotlib as fpl
import numpy as np

# data to plot
xs = np.linspace(0, 250, 10_000)
sine = np.sin(xs) * 20
cosine = np.cos(xs) * 20

plot = fpl.GridPlot((1, 2))

# sines and cosines
sines = [sine] * 1_00
cosines = [cosine] * 1_00

# make line stack
line_stack = plot[0, 0].add_line_stack(sines + cosines, separation=50)

# make selector
stack_selector = line_stack.add_linear_region_selector(padding=200)

zoomed_line_stack = plot[0, 1].add_line_stack([zoomed_init] * 2_000, separation=50, name="zoomed")
    
def update_zoomed_stack(ev):
    """update the zoomed subplots"""
    zoomed_data = stack_selector.get_selected_data()
    
    for i in range(len(zoomed_data)):
        data = interpolate(zoomed_data[i], axis=1)
        zoomed_line_stack.graphics[i].data = data
        
    plot[0, 1].auto_scale()


stack_selector.selection.add_event_handler(update_zoomed_stack)
plot.show(sidecar=False)